In [1]:
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedShuffleSplit

Some useful function

In [4]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated
    
def get_sampler(idx): # Return data sampler given pre_separated indexes
    train_idx, valid_idx, test_idx = idx
    sampler = {}
    sampler["train"] = SubsetRandomSampler(train_idx)
    sampler["val"] = SubsetRandomSampler(valid_idx)
    sampler["test"] = SubsetRandomSampler(test_idx)
    dataset_sizes = {"train": len(train_idx),"val": len(valid_idx),"test": len(test_idx)}
    return sampler, dataset_sizes
def get_idx_label(dataset): # Get label and their corresponding indexes 
    index = []
    label = []
    for i, data in enumerate(dataset):
        t,l = data
        index.append(i)
        label.append(l)
    return np.array(index), np.array(label)

Training a model 

In [5]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        # Each epoch has a training and validation phase
        for phase in ["train","val"]:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode



            # Iterate over data.
            running_loss = 0.0
            running_corrects = 0
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                #print(phase)
                #imshow(inputs[0])

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    #print(outputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
                # statistics
                #if i == 4:
                    #running_loss += loss.item() * inputs.size(0)
                    #running_corrects += torch.sum(preds == labels.data)

            #
            #

            #print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                #phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        #print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

Loading Dataset

In [6]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(234),
        transforms.RandomRotation(50),
        transforms.CenterCrop(224),
        #transforms.RandomAffine(0, translate=None, scale=None, shear=50, resample=False, fillcolor=0),
        #transforms.RandomResizedCrop(224),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(234),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(234),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
data_dir = '/Users/scraed/Downloads/Raphael-jpg'
image_datasets = {x: datasets.ImageFolder(data_dir,
                                          data_transforms[x])
                  for x in ['train', 'val', 'test']}

In [7]:
# Get the labels and corresponding indexes, ready for spliting
X,y = get_idx_label(image_datasets['train'])
# Cross validation iterator from scikit learn
kf = KFold(n_splits=22)
kf.get_n_splits(X)
# Spliter from scikit learn making labels are balanced when doing spliting
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.25)

In [8]:
Test_results = []
# Iteration on Leave one out cross validation
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    # Iteration on splited train val (Actually only run once, corresponds to n_splits=1 in StratifiedShuffleSplit)
    for t_idx, v_idx in sss.split( train_index, y[train_index] ):
        train = train_index[t_idx]; val = train_index[v_idx]
        print("TRAIN:", train, "Val:", val)
        
        
        # Preparing sampler
        samplers, dataset_sizes = get_sampler( (train,val,test_index) )
        dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, sampler=samplers[x],num_workers=4)
              for x in ['train', 'val', 'test']}
        
        
        # Initialize models
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        model_conv = torchvision.models.resnet18(pretrained=True)
        # Fix parameters
        for param in model_conv.parameters():
            param.requires_grad = False
        # Renewing the last FC layer, Parameters of newly constructed modules have requires_grad=True by default
        num_ftrs = model_conv.fc.in_features
        model_conv.fc = nn.Linear(num_ftrs, 2)
        # Preparing to train
        model_conv = model_conv.to(device)
        
        # Loss
        criterion = nn.CrossEntropyLoss()
        # Optimizer
        optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9,weight_decay = 0)
        # Decay LR by a factor of 0.1 every 7 epochs
        exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)
        
        #Trainning
        model_conv = train_model(model_conv, criterion, optimizer_conv,
                         exp_lr_scheduler, num_epochs=10)
        
        #Evaluate on Test
        model_conv.eval()
        running_corrects = 0.
        for inputs, labels in dataloaders['test']:
            outputs = model_conv(inputs)
            _, preds = torch.max(outputs, 1)
            running_corrects += torch.sum(preds == labels.data)
        test_acc = running_corrects.double() / dataset_sizes['test']
        print("Test acc "+ str(test_acc.numpy() )) 
        Test_results.append(test_acc.numpy() )
        

TRAIN: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21] TEST: [0]
TRAIN: [10 21  3  6 11 15 19  2  7  9 14  8 16 13  4] Val: [20 17  1 12  5 18]
Epoch 0/9
----------
train Loss: 0.8016 Acc: 0.3333
val Loss: 0.9792 Acc: 0.5000
Epoch 1/9
----------
train Loss: 0.8553 Acc: 0.4667
val Loss: 0.9095 Acc: 0.5000
Epoch 2/9
----------
train Loss: 0.7364 Acc: 0.6000
val Loss: 0.8848 Acc: 0.5000
Epoch 3/9
----------
train Loss: 0.6590 Acc: 0.7333
val Loss: 0.8719 Acc: 0.3333
Epoch 4/9
----------
train Loss: 0.5490 Acc: 0.8667
val Loss: 0.8180 Acc: 0.5000
Epoch 5/9
----------
train Loss: 0.6264 Acc: 0.6000
val Loss: 0.8564 Acc: 0.5000
Epoch 6/9
----------
train Loss: 0.6987 Acc: 0.4667
val Loss: 0.9280 Acc: 0.5000
Epoch 7/9
----------
train Loss: 0.6150 Acc: 0.6667
val Loss: 0.9962 Acc: 0.6667
Epoch 8/9
----------
train Loss: 0.5830 Acc: 0.6667
val Loss: 0.9156 Acc: 0.6667
Epoch 9/9
----------
train Loss: 0.4297 Acc: 0.9333
val Loss: 0.8950 Acc: 0.6667
Training complete in 0m 46s
B

Test acc 0.0
TRAIN: [ 0  1  2  3  4  5  6  7  9 10 11 12 13 14 15 16 17 18 19 20 21] TEST: [8]
TRAIN: [14 19 21  4 16 12  3  9 15  7 11  6 20  5  2] Val: [ 1 10  0 13 18 17]
Epoch 0/9
----------
train Loss: 0.7991 Acc: 0.5333
val Loss: 0.5674 Acc: 0.6667
Epoch 1/9
----------
train Loss: 0.6886 Acc: 0.6000
val Loss: 0.5725 Acc: 0.6667
Epoch 2/9
----------
train Loss: 0.7127 Acc: 0.5333
val Loss: 0.5788 Acc: 0.6667
Epoch 3/9
----------
train Loss: 0.5676 Acc: 0.6667
val Loss: 0.5930 Acc: 0.6667
Epoch 4/9
----------
train Loss: 0.5068 Acc: 0.6000
val Loss: 0.5738 Acc: 0.6667
Epoch 5/9
----------
train Loss: 0.5812 Acc: 0.8667
val Loss: 0.6242 Acc: 0.5000
Epoch 6/9
----------
train Loss: 0.8005 Acc: 0.4667
val Loss: 0.5344 Acc: 0.8333
Epoch 7/9
----------
train Loss: 0.4394 Acc: 0.7333
val Loss: 0.5342 Acc: 0.6667
Epoch 8/9
----------
train Loss: 0.5174 Acc: 0.8000
val Loss: 0.5728 Acc: 0.5000
Epoch 9/9
----------
train Loss: 0.5341 Acc: 0.8000
val Loss: 0.6437 Acc: 0.6667
Training complet

val Loss: 1.0273 Acc: 0.5000
Training complete in 0m 45s
Best val Acc: 0.500000
Test acc 1.0
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 17 18 19 20 21] TEST: [16]
TRAIN: [13 14 15  3  1  7  2  9 20  0  5 19 18 10 21] Val: [ 6  4 11  8 17 12]
Epoch 0/9
----------
train Loss: 0.6887 Acc: 0.6000
val Loss: 0.7001 Acc: 0.5000
Epoch 1/9
----------
train Loss: 0.9288 Acc: 0.6000
val Loss: 0.8589 Acc: 0.5000
Epoch 2/9
----------
train Loss: 0.9396 Acc: 0.4667
val Loss: 0.6811 Acc: 0.6667
Epoch 3/9
----------
train Loss: 0.7515 Acc: 0.5333
val Loss: 0.5907 Acc: 0.6667
Epoch 4/9
----------
train Loss: 0.6023 Acc: 0.7333
val Loss: 0.8579 Acc: 0.5000
Epoch 5/9
----------
train Loss: 0.5724 Acc: 0.6667
val Loss: 0.6752 Acc: 0.5000
Epoch 6/9
----------
train Loss: 0.6016 Acc: 0.7333
val Loss: 0.6136 Acc: 0.5000
Epoch 7/9
----------
train Loss: 0.5039 Acc: 0.7333
val Loss: 0.6281 Acc: 0.5000
Epoch 8/9
----------
train Loss: 0.5848 Acc: 0.6667
val Loss: 0.6847 Acc: 0.5000
Epoch 9/9
------

In [15]:
cross_acc = np.average( np.array( Test_results ) )
cross_var = np.std( np.array( Test_results ))
print("Cross validation accuracy: " + str(cross_acc))
print("Cross validation variance: " + str(cross_var))

Cross validation accuracy: 0.590909090909
Cross validation variance: 0.491666083018
